[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/MagicDance-jupyter/blob/main/MagicDance_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/MagicDance
%cd /content/MagicDance
!pip install tensorboard==2.12.1 langdetect==1.0.9 omegaconf==2.3.0 diffusers==0.11.1 pytorch-lightning==2.1.3 clip==0.2.0

!apt -y install -qq aria2 unzip
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/MagicDance/resolve/main/control_sd15_ini.ckpt -d /content/MagicDance/pretrained_weights -o control_sd15_ini.ckpt
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/MagicDance/resolve/main/control_v11p_sd15_openpose.pth -d /content/MagicDance/pretrained_weights -o control_v11p_sd15_openpose.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/MagicDance/resolve/main/model_state-10000.th -d /content/MagicDance/pretrained_weights -o model_state-10000.th
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/MagicDance/resolve/main/model_state-110000.th -d /content/MagicDance/pretrained_weights -o model_state-110000.th
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/MagicDance/resolve/main/disco_test_set.zip -d /content/MagicDance/TikTok-v4 -o disco_test_set.zip
!unzip -qq /content/MagicDance/TikTok-v4/disco_test_set.zip -d /content/MagicDance/TikTok-v4
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/camenduru/MagicDance/resolve/main/pose_map_disco_test_set.zip -d /content/MagicDance/TikTok-v4 -o pose_map_disco_test_set.zip
!unzip -qq /content/MagicDance/TikTok-v4/pose_map_disco_test_set.zip -d /content/MagicDance/TikTok-v4

In [ ]:
!CUDA_VISIBLE_DEVICES=0 torchrun --master_port 10000 test_tiktok.py \
--model_config /content/MagicDance/model_lib/ControlNet/models/cldm_v15_reference_only_pose.yaml \
--num_train_steps 10 \
--img_bin_limit all \
--train_batch_size 1 \
--use_fp16 \
--control_mode controlnet_important \
--control_type body+hand+face \
--train_dataset tiktok_video_arnold \
--with_text \
--wonoise \
--local_image_dir /content/MagicDance/tiktok_test_log/image_log/magicdance \
--local_log_dir /content/MagicDance/tiktok_test_log/tb_log/magicdance \
--image_pretrain_dir /content/MagicDance/pretrained_weights/model_state-110000.th \
--init_path /content/MagicDance/pretrained_weights/control_sd15_ini.ckpt